참고: https://jeinalog.tistory.com/13

## 2-1. 전이학습

### 2-1-1. 전이학습의 개념
- 높은 정확도를 비교적 짧은 시간 내에 달성 가능
- 이미 학습되어 있는 패턴 활용
- 사전학습된 모델 이용 (내가 풀고자 하는 문제와 비슷하면서 사이즈가 큰 데이터로 이미 학습이 되어 있는 모델 import)

### 2-1-2. CNN, 합성곱 신경망
- Convolutional base: 합성곱층과 pooling층이 여러겹 쌓여있는 부분
    - 이미지로부터 특징 효과적으로 추출하는 것이 목표
- Classifier: 완전 연결 계층으로 이루어짐
    - 추출된 특징을 잘 학습해서 이미지에 알맞은 카테고리로 분류하는 것이 목표

### 2-1-3. 사전 학습된 모델을 내 프로젝트에 맞게 재정의
- 1. 원래 모델의 classifier 삭제
- 2. 내 목적에 맞는 classifer 추가
- 3. 파인튜닝 전략
    - 전체 모델 새로 학습: 이 경우에는 사전학습 모델의 구조만 사용한다. 모델을 완전히 새로 학습시켜야하므로, 큰 사이즈의 데이터셋과 좋은 컴퓨팅 연산 능력이 있을 때 적합
    - Convolutional base의 일부분은 고정시킨 상태로, 나머지 계층과 classifier만 새로 학습: 데이터셋의 크기에 따라 얼마나 많은 계층을 새로 학습시킬지 달라지는데, 데이터의 양이 많을수록 더 많이 새로 학습시키고, 데이터의 양이 적을수록 학습시키는 부분을 적게 한다.
    - Convolutional base는 고정시키고, classifier만 새로 학습시키는 것이다. 이 경우는 convolutional base는 건들지 않고 그대로 두면서 특징 추출 매커니즘으로 활용하고, classifier만 재학습시키는 방법이다. 컴퓨팅 연산능력이 부족하거나 데이터셋이 작을 때 고려

내가 사용할 모델은 ILSVRC에서 쓰인 모델로 큰 사이즈의 데이터셋인 반면, 새로 학습 시킬 데이터는 약 2만 장으로 비교적 매우 작다. 따라서 가져오는 모델은 이미지에서 특징을 추출하는 매커니즘으로 활용하고, Classifier만 새로 학습시키는 전략이 적절하다.

### 2-1-4. 전이학습 과정
1) 사전학습 모델 선택

2) 내 문제가 데이터 크기-유사성 그래프에서 어떤 부분에 속하는지 알아보기

3) 내 모델 fine-tuning

### 2-1-5. Classifer
- Convolutional base: 이미지로부터 특징 추출하는 부분
    - 낮은 레벨의 계층: input에 가까운 계층으로, 이미지에서 주로 일반적인(general) 특징을 추출
    - 높은 레벨의 계층: output에 가까운 계층으로, 보다 구체적이고 특유한 특징을 추출
- Classifier: 추출된 특징을 입력받아서 최종적으로 이미지 카테고리 결정하는 부분
    - Fully-connected layers: 완전 연결 계층을 쌓은 후 마지막에 소프트맥스 활성화함수(softmax activated layer) 계층을 놓는 것
    - Global average pooling, 평균 풀링: Fully-connected layer 대신에 평균 풀링계층을 추가하고, 그 결과값을 바로 소프트맥스 계층과 연결
    - Linear support vector machines, 선형 서포트 벡터 머신

## 2-2. 사전학습모델 VGG16

In [ ]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Create the base model from the pre-trained model VGG16
base_model = tf.keras.applications.VGG16(input_shape=IMG_SHAPE,  # VGG 모델 불러오기
                                         include_top=False,      # output에 가까운 높은 레벨에 있는 3개 FC 레이어는 제외하고 불러와야해서 False 옵션
                                         weights='imagenet')

In [ ]:
image_batch.shape # 학습 데이터 원래 사이즈 확인

# TensorShape([32, 160, 160, 3])
# [이미지 장수, (이미지 크기*크기), 채널 수]

In [ ]:
# 모델에 배치 넣기
feature_batch = base_model(image_batch)
feature_batch.shape

# TensorShape([32, 5, 5, 512])
# 이미지 개수 동일, 사이즈 감소, 채널 증가
# 특징 벡터!

In [ ]:
# 모델 구조
base_model.summary()

# dense 블록이 빠진 이유는 전이학습을 수해앟며 새로 학습시킬 것이기 때문이다.

### 2-3. VGG16 끝단에 classifier 레이어 붙여서 원하는 구조의 분류 모델
- VGG16이 출력하는 벡터의 shape: (32, 5, 5, 512): 3차원
- classifier를 구성하려면 fully connected 레이어로 구성해야 함: 반드시 1차원

In [13]:
# flatten
import numpy as np

image = np.array([[1, 2],
                  [3, 4]])

flattened_image = image.flatten()

print("Original image:\n", image)
print("Original image shape:", image.shape)
print()
print("Flattened image:\n", flattened_image)
print("Flattened image shape:", flattened_image.shape)

Original image:
 [[1 2]
 [3 4]]
Original image shape: (2, 2)

Flattened image:
 [1 2 3 4]
Flattened image shape: (4,)


#### 2-3-1. 2D Global average pooling
- 3차원의 벡터가 있을 때, 겹겹이 쌓여있는 2차원 배열을 평균을 취한 후 하나의 값으로 축소시키는 기법
- Global Average Pooling을 거치게 되면 (Height, Width, Depth)의 shape를 가지는 3차원 이미지의 shape가 (1, 1, Depth)로 바뀐다. 즉, (Depth)의 shape로, 1차원 벡터로 볼 수 있다.
-  VGG16이 출력한 벡터가 [32, 5, 5, 512]였으니, Global Average Pooling을 취하면 벡터의 shape가 [32, 512]가 된다.

In [14]:
# Global Average Pooling 계층을 만드는 코드
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

In [ ]:
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)
# (32, 512)

In [ ]:
# Dense 레이어 붙여주면 새로운 classifier 완성
dense_layer = tf.keras.layers.Dense(512, activation='relu')
prediction_layer = tf.keras.layers.Dense(2, activation='softmax')

# feature_batch_averag가 dense_layer를 거친 결과가 다시 prediction_layer를 거치게 되면
prediction_batch = prediction_layer(dense_layer(feature_batch_average))  
print(prediction_batch.shape)

In [ ]:
base_model.trainable = False # 학습 안 시킬거라서 train 변수 off 
# VGG16 base_model 에 입력되어 특징이 추출된 다음, 특징벡터는 global_average_layer를 거쳐 마지막에 prediction_layer까지 통과하며 강아지인지, 고양이인지 예측

### 2-4. 최종 모델

In [ ]:
model = tf.keras.Sequential([
  base_model,  # VGG16 내 레이어들 간결하게 표현
  global_average_layer,
  dense_layer,
  prediction_layer
])

model.summary()

#### 2-4-1. VGG16 기반 이미지 분류와 직접 만든 분류 모델 결과 비교

In [ ]:
# 모델 학습
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

In [ ]:
# 아직 학습되지 않은 상태
validation_steps=20
loss0, accuracy0 = model.evaluate(validation_batches, steps = validation_steps)

print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

In [ ]:
EPOCHS = 5   # 이번에는 이전보다 훨씬 빠르게 수렴되므로 5Epoch이면 충분

history = model.fit(train_batches,
                    epochs=EPOCHS,
                    validation_data=validation_batches)

In [ ]:
# 학습 과정 그래프
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
# 샘플 예측 결과
for image_batch, label_batch in test_batches.take(1):
    images = image_batch
    labels = label_batch
    predictions = model.predict(image_batch)
    pass

predictions # 확률값 0과 1 사이

In [ ]:
# prediction 값들을 실제 추론한 라벨(고양이:0, 강아지:1)로 변환
import numpy as np
predictions = np.argmax(predictions, axis=1)
predictions

In [ ]:
plt.figure(figsize=(20, 12))

for idx, (image, label, prediction) in enumerate(zip(images, labels, predictions)):
    plt.subplot(4, 8, idx+1)
    image = (image + 1) / 2
    plt.imshow(image)
    correct = label == prediction
    title = f'real: {label} / pred :{prediction}\n {correct}!'
    if not correct:
        plt.title(title, fontdict={'color': 'red'})
    else:
        plt.title(title, fontdict={'color': 'blue'})
    plt.axis('off')

In [ ]:
# 정확도 확인
count = 0
for image, label, prediction in zip(images, labels, predictions):
    correct = label == prediction
    if correct:
        count = count + 1

print(count / 32 * 100) # 약 95% 내외

### 2-5. 모델을 save 하고, 다시 load 해와서 사용하는 방법
- 저장: save_weights

In [ ]:
import os

checkpoint_dir = os.getenv("HOME") + "/workplace/aiffel/Exploration/04. 이미지 분류/cat_vs_dog/checkpoint"
checkpoint_file_path = os.path.join(checkpoint_dir, 'checkpoint')

if not os.path.exists('checkpoint_dir'):
    os.mkdir('checkpoint_dir')
    
model.save_weights(checkpoint_file_path)     # checkpoint 파일 생성

if os.path.exists(checkpoint_file_path):
  print('checkpoint 파일 생성 OK!!')

### 2-6. 학습된 모델에 원하는 이미지를 입력해 예측 결과 확인

In [ ]:
# 이미지 저장
img_dir_path = os.getenv("HOME") + "/aiffel/cat_vs_dog/images"
os.path.exists(img_dir_path)

In [ ]:
# 텐서 플로우 모델에 입력
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
# 파라미터로 이미지 사이즈 설정
IMG_SIZE = 160
dog_image_path = os.path.join(img_dir_path, 'my_dog.jpeg')

dog_image = load_img(dog_image_path, target_size=(IMG_SIZE, IMG_SIZE))
dog_image

In [ ]:
# 배열자료형 변환
dog_image = img_to_array(dog_image).reshape(1, IMG_SIZE, IMG_SIZE, 3)
dog_image.shape

In [ ]:
# 고양이라면 [1.0, 0.0], 강아지라면 [0.0, 1.0]에 가까운 확률 분포가 예측
prediction = model.predict(dog_image)
prediction

In [ ]:
# 예측 함수
def show_and_predict_image(dirpath, filename, img_size=160):
    filepath = os.path.join(dirpath, filename)
    image = load_img(filepath, target_size=(img_size, img_size))
    plt.imshow(image)
    plt.axis('off')
    image = img_to_array(image).reshape(1, img_size, img_size, 3)
    prediction = model.predict(image)[0]
    cat_percentage = round(prediction[0] * 100)
    dog_percentage = round(prediction[1] * 100)
    print(f"This image seems {dog_percentage}% dog, and {cat_percentage}% cat.")

In [ ]:
# 함수에 이미지 넣기
filename = 'my_dog.jpeg'

show_and_predict_image(img_dir_path, filename)

In [ ]:
filename = 'my_cat.jpeg'

show_and_predict_image(img_dir_path, filename)